# Importing Dependencies

In [87]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.schema import Document
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [64]:
load_dotenv()

True

In [67]:
parser = StrOutputParser()

# Indexing (Document Ingestion)

In [68]:
from groq import Groq
import glob

def transcribe_audio_groq_chunks(chunk_dir="."):
    """
    Transcribes all chunk_XXX.mp3 files using Groq Whisper
    and returns a single combined transcript string.
    """
    client = Groq()
    full_text = []

    # Ensure correct order
    chunk_files = sorted(glob.glob(f"{chunk_dir}/chunk_*.mp3"))

    for chunk in chunk_files:
        print(f"Transcribing {chunk}...")

        with open(chunk, "rb") as audio_file:
            result = client.audio.transcriptions.create(
                file=audio_file,
                model="whisper-large-v3-turbo"
            )

        full_text.append(result.text)

    return "\n".join(full_text)


In [ ]:

transcribed_text = transcribe_audio_groq_chunks(
    chunk_dir="/Users/rushikeshankaikar/Documents/Langchain/audio"
)

In [70]:
print(transcribed_text)

 Welcome to the complete Python Mastery course. In this course, you're going to learn everything about Python from basics to more advanced concepts. So by the end of the course, you'll be able to confidently use Python for AI, machine learning, web development, and automation. If you have been looking for a comprehensive, easy to follow, well-organized, and practical course that takes you from zero to hero, this is the right Python course for you. You don't need any prior knowledge of Python to get started. I will explain everything step by step in simple terms so you can build a solid foundation. I'm Mosh Hamadani, a software engineer with over 20 years of experience, and I've taught millions how to code and become professional software engineers through my YouTube channel and online school CodeWithMosh.com. If you're new here, make sure to subscribe as I upload new videos all the time. Now, let's jump in and get started. In this course, you're going to learn everything you need to ge

# Chunking

In [88]:
docs = [
    Document(
        page_content=transcribed_text,
        metadata={
            "source": "youtube",
            "video_id": "K5KVEU3aaeQ"
        }
    )
]

In [71]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
)

In [72]:
chunks = splitter.split_text(transcribed_text)

In [77]:
print(chunks[1])

CodeWithMosh.com. If you're new here, make sure to subscribe as I upload new videos all the time. Now, let's jump in and get started. In this course, you're going to learn everything you need to get started with Python. Just be aware that I've designed this course for beginners. So if you have some programming experience, check out my other Python course for developers. You can see the link on the top right corner of this video. So Python is the world's fastest growing and most popular programming language, not just amongst software developers, but also amongst mathematicians, data analysts, scientists, accountants, network engineers, and even kids. In fact, it's the ideal programming language to learn first. But what makes Python so special? Here are six reasons. With Python, you can solve complex problems in less time with fewer lines of code than many other languages. That's why huge companies like Google, Spotify, Dropbox, and Facebook have embraced this beautiful and powerful


# 1. Embedding and Vector Storing

In [99]:
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [100]:
vector_store = FAISS.from_texts(chunks, embedding_model)

# 2. Retrival

In [91]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [92]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13f631f10>, search_kwargs={'k': 4})

In [98]:
response = retriever.invoke('What is Python?')
print(response[0].page_content)

CodeWithMosh.com. If you're new here, make sure to subscribe as I upload new videos all the time. Now, let's jump in and get started. In this course, you're going to learn everything you need to get started with Python. Just be aware that I've designed this course for beginners. So if you have some programming experience, check out my other Python course for developers. You can see the link on the top right corner of this video. So Python is the world's fastest growing and most popular programming language, not just amongst software developers, but also amongst mathematicians, data analysts, scientists, accountants, network engineers, and even kids. In fact, it's the ideal programming language to learn first. But what makes Python so special? Here are six reasons. With Python, you can solve complex problems in less time with fewer lines of code than many other languages. That's why huge companies like Google, Spotify, Dropbox, and Facebook have embraced this beautiful and powerful


# 3.Augmentation

In [101]:
model = ChatGroq(
    model='openai/gpt-oss-120b',
    temperature=0.5
)

In [102]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [103]:
question          = "Is the topic of Loops discussed in this video? If yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [104]:
print(retrieved_docs)

[Document(page_content="because it's indented four times this statement itself is a loop So what we have inside of this loop will be executed three times In the first iteration X is zero because we are still in the first iteration of the outer loop and Y is also zero because we are in the first iteration of the inner loop That is why we get zero and zero Now we go to the second iteration of this inner loop in this iteration Y will be one whereas X is still zero That is why we get zero and one and similarly in the third iteration of our inner loop will get 0 and 2 in the terminal now we're done with the execution of the inner loop So the control moves back to our outer loop here will be in the second iteration So X will be 1 and then we start here again So we have to execute this inner loop three times in the first situation Y will be 0 and X is 1 so here we have 1 and 0 then we'll get 1 and 1 and 2 you got the point so this is all about nested loops so you have learned how to use for l

In [105]:
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [108]:
final_prompt = prompt.invoke({
    'question':question,
    'context': context
})

In [109]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      because it's indented four times this statement itself is a loop So what we have inside of this loop will be executed three times In the first iteration X is zero because we are still in the first iteration of the outer loop and Y is also zero because we are in the first iteration of the inner loop That is why we get zero and zero Now we go to the second iteration of this inner loop in this iteration Y will be one whereas X is still zero That is why we get zero and one and similarly in the third iteration of our inner loop will get 0 and 2 in the terminal now we're done with the execution of the inner loop So the control moves back to our outer loop here will be in the second iteration So X will be 1 and then we start here again So we have to execute this inner loop three times in the first situati

# 4. Generation

In [110]:
answer = model.invoke(final_prompt)

In [111]:
print(answer)

content='Yes. The video covers loops. It explains:\n\n* **For loops** – how they iterate over iterable objects, using the `range` function to generate a sequence of numbers, and how nested for‑loops work (showing the X‑Y example with multiple inner‑loop iterations).  \n* **The `range` object** – that `range()` returns an object of type\u202f`range`, a complex type used with for‑loops.  \n* **While loops** – using a condition (`number > 0`) to repeat statements until the condition becomes false, with examples of dividing a number by two each iteration.  \n* **Practical use of loops** – using a for‑loop with `range(3)` to retry a task (e.g., sending a message) a set number of times instead of repeating many lines of code.  ' response_metadata={'token_usage': {'completion_tokens': 283, 'prompt_tokens': 997, 'total_tokens': 1280, 'completion_time': 0.589602826, 'completion_tokens_details': {'reasoning_tokens': 101}, 'prompt_time': 0.046765163, 'prompt_tokens_details': None, 'queue_time': 0

# Chain formation

In [113]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableSequence, RunnableLambda

In [114]:
def format_docs(retrieved_docs):
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context

In [116]:
parallel_chain = RunnableParallel({
    'question': RunnablePassthrough(),
    'context': retriever | RunnableLambda(format_docs)
})

In [ ]:
final_chain = parallel_chain | prompt | model | parser

In [120]:
final_chain.invoke('Can you summarize the vedio ?')

'The video is the first two‑hour segment of a complete Python mastery course. It introduces basic Python concepts, starting with variables and then moves on to Boolean logic and operators (and, or, not). The instructor explains how Boolean expressions are evaluated with short‑circuiting, using a loan‑eligibility example (“high income and good credit and not a student”). He also mentions that Python can compile to Java bytecode, allowing Java code to be imported via JSON. Throughout, he encourages viewers to like, share, and check the description for a link to the full course, which covers beginner to advanced topics such as machine learning, web development, and automation, plus hands‑on projects and quizzes.'